# SparkR
<!-- requirement: small_data/stackexchange -->

Accessing Spark from an R interface.
[SIGMOD paper](https://people.csail.mit.edu/matei/papers/2016/sigmod_sparkr.pdf)

This demo uses `responses.csv`, the results of the Stack Exchange 2016 Developer Survey. Since it is ~ 50 MB, you will need to download this file (s3://dataincubator-course/stackexchange/responses.csv) into `small_data/stackexchange/` to proceed.

In [ ]:
library(dplyr)
library(magrittr)
library(ggplot2)

In [ ]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sparkR.session(master = "local[*]", sparkConfig = list(spark.driver.memory = "2g"))

*Note*: We loaded SparkR last so it could have the last mask.

In [ ]:
# Telling Spark to use the local file system
localpath <- function(path) {
    return(paste("file://", getwd(), "/", path, sep = ""))
}

In [ ]:
df1 <- as.DataFrame(faithful)

In [ ]:
df1 %>% str

In [ ]:
df1 %>% printSchema

In [ ]:
df <- read.df(localpath("small_data/stackexchange/responses.csv.gz"),
              "csv",
              header = "true",
              inferSchema = "true",
              na.strings = "NA") %>%
    cache

In [ ]:
# You can also read directly from S3 with authentication set up properly
# Make absolutely sure to `cache` in this case or you will be in for an unpleasant surprise
# df <- read.df("s3n://dataincubator-course/stackexchange/responses.csv",
#               "csv",
#               header = "true",
#               inferSchema = "true",
#               na.strings = "NA") %>%
#     cache

In [ ]:
df %>% str

In [ ]:
df %>% printSchema

In [ ]:
df %>% head

In [ ]:
df %>% select("self_identification") %>% head

In [ ]:
# Nope:
# df %>% filter(collector == "Facebook") %>% select(self_identification) %>% head
df %>% filter(df$collector != "Facebook") %>% select(df$self_identification) %>% head

In [ ]:
?summary

In [ ]:
system.time(stats <- df %>% summary %>% cache)

In [ ]:
system.time(stats %>% select("summary", "salary_midpoint"))

In [ ]:
stats %>% select("summary", "salary_midpoint") %>% collect

### Aggregation functions

* avg
* min
* max
* sum
* countDistinct
* sumDistinct

In [ ]:
df %>% select(avg(df$salary_midpoint)) %>% collect

### Ordering

In [ ]:
df %>% select("country", "age_range", "salary_range") %>% arrange(desc(df$salary_midpoint)) %>% head

### Filtering

In [ ]:
salaries <- df %>% select("salary_midpoint", "age_range")

In [ ]:
salaries %>% head

In [ ]:
salaries %<>% filter("salary_midpoint > 0 and age_range != 'NA'")

In [ ]:
salaries %>% head

### Grouping

Combine groupby with aggregation or summary.

In [ ]:
# age_groups <- agg(
#     groupBy(salaries, "age_range"), 
#     number = n(salaries$salary_midpoint),
#     avg_sal = avg(salaries$salary_midpoint), 
#     max_sal = max(salaries$salary_midpoint),
#     min_sal = min(salaries$salary_midpoint)
# )

In [ ]:
age_groups <- salaries %>%
    groupBy("age_range") %>% 
    summarize(
        number = n(salaries$salary_midpoint),
        avg_sal = avg(salaries$salary_midpoint), 
        max_sal = max(salaries$salary_midpoint),
        min_sal = min(salaries$salary_midpoint))

In [ ]:
age_groups %>% head

In [ ]:
sorted_ages <- age_groups %>% arrange(asc(age_groups$age_range)) %>% collect
sorted_ages

In [ ]:
age_groups %>% str

In [ ]:
?factor

In [ ]:
ages_vec <- unique(collect(salaries)$age_range) %>% sort
ages_vec

Let's make sure to apply this to the in-memory data frame, not the SparkDataFrame.

In [ ]:
sorted_ages$age_range <- factor(
    sorted_ages$age_range,
    ages_vec
)
sorted_ages %>% str

In [ ]:
resorted_ages <- sorted_ages[order(sorted_ages$age_range),]
resorted_ages

In [ ]:
plot <- ggplot(data = head(resorted_ages, -1), aes(x=age_range, y=avg_sal, group=1))
plot + geom_line() + geom_point() + ylab("Average salary") +xlab("Age range")

## Selecting using R and SQL

In [ ]:
salaries %>% select(salaries$salary_midpoint / 1000) %>% head

In [ ]:
salaries %>% selectExpr("(salary_midpoint / 1000) as Salary_K") %>% head

In [ ]:
createOrReplaceTempView(df, "data")

In [ ]:
highpaid <- sql("select occupation, star_wars_vs_star_trek from data where salary_midpoint > 200000 and star_wars_vs_star_trek != 'NA'")

In [ ]:
highpaid %>% head

**Exercise**: Obviously our data is somewhat messy. Clean it up.

In [ ]:
df %>% subset(df$salary_midpoint > 200000, c("occupation", "age_range")) %>% head

Some other familiar operations to try:
* nrow, ncol
* rbind, cbind

## Modeling

In [ ]:
titanic <- as.data.frame(Titanic)

In [ ]:
titanicDF <- createDataFrame(titanic[titanic$Freq > 0, -5])
nbDF <- titanicDF
nbTestDF <- titanicDF

In [ ]:
titanicDF %>% head

In [ ]:
nbModel <- spark.naiveBayes(nbDF, Survived ~ Class + Sex + Age)

In [ ]:
nbModel %>% summary

In [ ]:
nbPredictions <- predict(nbModel, nbTestDF)
nbPredictions %>% showDF

More examples can be found on [GitHub](https://github.com/apache/spark/blob/master/examples/src/main/r/ml/ml.R).

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*